In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import random
from IPython.display import clear_output
import os.path

ModuleNotFoundError: No module named 'bs4'

In [82]:
length = 1000
headers = ['word', 'sen1', 'sen2', 'sen3', 'choice1', 'choice2', 'choice3']
url = 'https://sentence.yourdictionary.com/'

,0
0,entomology
1,hirsute
2,cadence
3,ample
4,circumspect
...,...
995,equanimity
996,pronate
997,swear-word
998,jettatura


In [ ]:
def get_progress(data):
    for i in data.index:
#        if data['sen1'][i]==None:
        if (type(data['sen1'][i])==float) or (data['sen1'][i]==None):
            return i

In [ ]:
def read_raw(url='https://sentence.yourdictionary.com/',length=1000):
    df = pd.read_csv('data_raw.csv', on_bad_lines='skip', header=None,sep='\t')
    data = df.T.head(1).T
    data = data.replace({'<br>': ''},regex=True)
    data = data[data[0].str.contains(' ')==False]
    data = data.sample(n=length, random_state=2022).reset_index(drop=True)
    data['sen1'] = None
    data['sen2'] = None
    data['sen3'] = None
    data['choice1'] = None
    data['choice2'] = None
    data['choice3'] = None
    data = data.rename(columns={0:'word'})
    data.head()
    return data

In [ ]:
def read_sentence(word,url = 'https://sentence.yourdictionary.com/'):
    page = requests.get(url+word)
    soup = BeautifulSoup(page.content,'html.parser')
    sentences = soup.find_all('span', {'class':"sentence-item__text"})
    #meaning = soup.find_all('div', {'class':"definition flex-align-self-center"})
    sens = []
    print('--------')
    print(f'Current word: {word}')
    for sen in sentences:
        sen = str(sen).split('"">')[1].split('</span>')[0].replace('<mark>','').replace('</mark>','')
        print(sen)
        passed = input('Do you want to use this sentence? Type y or n: ')
        if passed == 'y':
            sens.append(sen)
        if len(sens) == 3:
            break
        print()
    clear_output()    
    return sens
    
def ask_for_choice(word):
    print(f'The word is {word}. Please enter 3 other words:\n')
    word1 = input('Word 1: ')
    word2 = input('Word 2: ')
    word3 = input('Word 3: ')
    choices = [word,word1,word2,word3]
    random.Random(2022).shuffle(choices)
    clear_output()
    return choices

In [ ]:
def build_word(data, i, url = 'https://sentence.yourdictionary.com/'):
    sent = read_sentence(data['word'][i])
    if len(sent) == 0:
        data = data.drop(i)
        data = data.reset_index(drop=True)
    else:
        data['sen1'][i] = sent[0]
        data['sen2'][i] = sent[1]
        data['sen3'][i] = sent[2]
        c = ask_for_choice(data['word'][i])
        data['choice1'][i] = c[1]
        data['choice2'][i] = c[2]
        data['choice3'][i] = c[3]
    return data

In [2]:
def main():
    if os.path.isfile('data.csv'): # if there is record
        print('Start with progress')
        data = pd.read_csv('data.csv', index_col=0)
        data = data.reset_index(drop=True)
    else: # if there is no previous record
        print('Start from scratch')
        data = read_raw()
    flag = True
    while flag:
        passed = input('Do you want to continue? y or n: ')
        if passed == 'y':
            i = get_progress(data)
            data = build_word(data, i)
            data.to_csv('data.csv')
        else:
            flag = False

In [105]:
def build_dataset(words,url='https://sentence.yourdictionary.com/',length=1000):
    dataset_dict = {}
    for word in words[0]:
        sentences = read_sentence(word)
        if len(sentences) == 0:
            continue
        choices = ask_for_choice(word)
        dataset_dict[word] = {'sentences':sentences,'choices':choices}
        if len(dataset_dict) > length:
            break
    return dataset_dict

def read_sentence(word,url = 'https://sentence.yourdictionary.com/'):
    page = requests.get(url+word)
    soup = BeautifulSoup(page.content,'html.parser')
    sentences = soup.find_all('span', {'class':"sentence-item__text"})
    #meaning = soup.find_all('div', {'class':"definition flex-align-self-center"})
    sens = {}
    idx = 0
    print('--------')
    print(f'Current word: {word}')
    for sen in sentences:
        sen = str(sen).split('"">')[1].split('</span>')[0].replace('<mark>','').replace('</mark>','')
        print(sen)
        passed = input('Do you want to use this sentence? Type y or n: ')
        if passed == 'y':
            sens[idx] = sen
            idx += 1
        if len(sens) == 3:
            break
        print()
    clear_output()    
    return sens
    
def ask_for_choice(word):
    print(f'The word is {word}. Please enter 3 other words:\n')
    word1 = input('Word 1: ')
    word2 = input('Word 2: ')
    word3 = input('Word 3: ')
    choices = [word,word1,word2,word3]
    random.Random(2022).shuffle(choices)
    clear_output()
    return choices

In [106]:
main()

--------
Current word: entomology
The entomology of the island, however, is very rich, particularly in respect of Lepidoptera.


KeyboardInterrupt: Interrupted by user